In [1]:
## Import the BSP and Remapping components of the WM_Methods package
from WM_Methods import Remapping
## Other required packages for calculations and plotting
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Thermal expansion, haline contraction and true scale factor
alph = 1.7657*10**-4
bet = 7.5544*10**-4

volnorming = 10**15 #normalising coeffcients
areanorming = 10**12 #normalising coeffcients
ST_scale=bet/alph

# Establish basic constants 
yr2sec = 365.25*24*60*60
Cp=4000
rho=1024
S0=35

# Range of years of which 'early' and 'late' are defined
dyrs = 10
init_early = 1970
init_late = 2005
Early_period = (np.array([init_early,init_early+dyrs]) - 1970)*12
Late_period = (np.array([init_late,init_late+dyrs]) - 1970)*12

In [3]:
## LOAD T and S data from a gridded observations (e.g., we use EN4 here)
## For the purposes of the tutorial we only select the first 3 months of the data
data = xr.open_mfdataset('~/UNSW_work/EN4_Data/EN_data/EN4_CT_SA_*')

T = data.Cons_Temp.isel(time=slice(Early_period[0],Early_period[1])).mean('time', keepdims=True)
S = data.Abs_Sal.isel(time=slice(Early_period[0],Early_period[1])).mean('time', keepdims=True)

In [4]:
display(T)

<xarray.DataArray 'Cons_Temp' (time: 1, depth: 42, lat: 173, lon: 360)>
dask.array<getitem, shape=(1, 42, 173, 360), dtype=float32, chunksize=(1, 42, 173, 360), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float32 1.0 2.0 3.0 4.0 5.0 ... 356.0 357.0 358.0 359.0 360.0
  * depth    (depth) float32 5.022 15.08 25.16 ... 4.752e+03 5.051e+03 5.35e+03
  * lat      (lat) float32 -83.0 -82.0 -81.0 -80.0 -79.0 ... 86.0 87.0 88.0 89.0
Dimensions without coordinates: time

In [5]:
EN4_BSP_data = xr.open_mfdataset('BSP_processed/BSP_EN4_TS_*.nc')

## Early Period
Part_early = EN4_BSP_data.Partitions.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time', keepdims=True)
SA_early =  EN4_BSP_data.S_mean.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time', keepdims=True)
CT_early = EN4_BSP_data.T_mean.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time', keepdims=True)
V_early = EN4_BSP_data.V_sum.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time', keepdims=True)
A_early = EN4_BSP_data.A_sum.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time', keepdims=True)

Basins = EN4_BSP_data.Basin.values

In [6]:
print(Part_early.shape)

(9, 1, 128, 4)


In [7]:
mask_EN4_xr = xr.open_mfdataset('mask_EN4.nc')
mask_EN4 = (mask_EN4_xr.__xarray_dataarray_variable__.values)


In [8]:
Opt_result = xr.open_mfdataset('Optimal_result_EN4.nc')
dT_mixing = Opt_result.dT_mixing.values.reshape(Basins.size,SA_early.shape[-1])
dS_mixing = Opt_result.dS_mixing.values.reshape(Basins.size,SA_early.shape[-1])
dS_adj = Opt_result.dS_adjustment.values.reshape(Basins.size,SA_early.shape[-1])
dT_adj = Opt_result.dT_adjustment.values.reshape(Basins.size,SA_early.shape[-1])

## Remove NaNs from the visualisation
dT_mixing[np.isnan(dT_mixing)] = 0
dS_mixing[np.isnan(dS_mixing)] = 0
dS_adj[np.isnan(dS_adj)] = 0
dT_adj[np.isnan(dT_adj)] = 0

In [12]:
tree_depth = int(np.log2(SA_early.shape[-1]))

## Ensure tracers match the requirements of the remap_3D function by adding a time dimension
dT_mixing_3D = np.repeat(dT_mixing[np.newaxis,:,:], T.time.size, axis=0)
dS_mixing_3D = np.repeat(dS_mixing[np.newaxis,:,:], T.time.size, axis=0)
dS_adj_3D = np.repeat(dS_adj[np.newaxis,:,:], T.time.size, axis=0)
dT_adj_3D = np.repeat(dT_adj[np.newaxis,:,:], T.time.size, axis=0)

print(Part_early.shape)
## This (faster) remapping code maps a given tracer of shape (time, bin #) onto Eulerian space
dT_mixing_Eulerian = 0
dS_mixing_Eulerian = 0
dS_adj_Eulerian = 0
dT_adj_Eulerian = 0
for i in range(Basins.size):
    tmp = Remapping.remap_tracer(T,S, Part_early[i,:,:,:], dT_mixing_3D[:,i,:], depth=tree_depth, interp = False)*mask_EN4[i,:,:,:]
    dT_mixing_Eulerian = dT_mixing_Eulerian+tmp

(9, 1, 128, 4)


100%|██████████| 1/1 [00:05<00:00,  5.26s/it]


100%|██████████| 1/1 [00:04<00:00,  4.67s/it]


100%|██████████| 1/1 [00:04<00:00,  4.31s/it]


100%|██████████| 1/1 [00:04<00:00,  4.38s/it]


100%|██████████| 1/1 [00:04<00:00,  4.50s/it]


100%|██████████| 1/1 [00:04<00:00,  4.29s/it]


100%|██████████| 1/1 [00:04<00:00,  4.31s/it]


100%|██████████| 1/1 [00:04<00:00,  4.30s/it]


100%|██████████| 1/1 [00:04<00:00,  4.52s/it]


In [16]:
dT_mixing_Eulerian.isel(depth=0).plot()

KeyboardInterrupt: 